In [4]:
import pandas as pd
from itertools import combinations as comb
import csv

def fetch_synergies_from_file(drug_ids, csv_file_path="drug_data.csv"):
    """Fetch and return synergy scores for analysis from a CSV file."""
    synergy_results = {}
    drug_data = pd.read_csv(csv_file_path)

    pairs = comb(drug_ids, 2)  # Generate all unique pairs from the given drug IDs

    for id1, id2 in pairs:
        # Find the combination in the dataframe
        row = drug_data[(drug_data['idDrugA'].isin([id1, id2])) & (drug_data['idDrugB'].isin([id1, id2]))]
        if not row.empty:
            scores = {
                'bliss': row.iloc[0]['bliss'],
                'zip': row.iloc[0]['zip'],
                'loewe': row.iloc[0]['loewe'],
                'hsa': row.iloc[0]['hsa']
            }
        else:
            scores = "Combination Data Not Found"
        
        synergy_results[(id1, id2)] = scores

    return synergy_results

import numpy as np

def analyze_combination_synergy(synergy_data):
    """Analyze synergy focusing on Loewe scores, with immediate checks for ZIP and HSA."""
    overall_synergy_responses = {}
    any_significant_antagonism = False
    any_significant_synergy = False

    for pair, scores in synergy_data.items():
        loewe = scores['loewe']
        zip_score = scores['zip']
        hsa = scores['hsa']
        
        # Immediate check for significant antagonism based on ZIP
        if zip_score < 0.05:  # Example threshold for significant ZIP score indicating strong antagonism
            any_significant_antagonism = True
            break  # Exit the loop early if any pair is significantly antagonistic
        
        # Check for very high HSA indicating strong synergy
        if hsa > 20:  # Example threshold for HSA indicating strong synergy
            any_significant_synergy = True

        # Evaluate Loewe score for synergy or antagonism
        if loewe > 0:
            overall_synergy_responses[pair] = "Antagonistic"
        elif loewe < 0:
            overall_synergy_responses[pair] = "Synergistic"
        # Additive effects (Loewe = 1) do not directly imply synergy or antagonism

    # Final overall synergy evaluation
    if any_significant_antagonism:
        overall_result = "Antagonistic"
    elif any_significant_synergy and not any_significant_antagonism:
        overall_result = "Synergistic"
    else:
        # If no significant synergy or antagonism, defer to the balance of Loewe scores
        overall_result = "Synergistic" if all(response == "Synergistic" for response in overall_synergy_responses.values()) else "Antagonistic"

    return overall_result


def save_overall_responses_to_csv(overall_responses, output_csv_path):
    """Save the overall synergy responses of drug combinations to a CSV file."""
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Drug Combination", "Overall Synergy Response"])
        for combination, response in overall_responses.items():
            writer.writerow([', '.join(map(str, combination)), response])

def read_drug_combinations(input_file_path):
    """Read drug combinations from a CSV file."""
    combinations = []
    with open(input_file_path, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Convert the string of drug IDs into a list of integers
            drug_ids = [int(id.strip()) for id in row['Drug Combination'].split(',')]
            combinations.append(drug_ids)
    return combinations


In [5]:
# Process
input_file_path = "valid_drug_combinations.csv"  # Adjust this to your actual CSV file path
output_csv_path = "synergy_test_results.csv"

combinations = read_drug_combinations(input_file_path)
print(combinations)
overall_responses = {}

[[52, 45], [27, 69], [76, 106], [68, 102, 2], [102, 45], [31, 1, 45], [102, 1, 11], [74, 52], [2, 75], [65, 107], [63, 24], [33, 52], [106, 74, 65], [63, 52], [33, 2, 53], [2, 44], [97, 332, 88], [102, 65], [44, 105, 88], [14, 68, 52], [87, 52], [84, 88, 69], [33, 332, 1], [11, 97, 102], [103, 72], [92, 62], [84, 68], [24, 105, 69], [20, 34, 92], [1, 24], [103, 88, 44], [72, 36, 2], [332, 33, 88], [24, 76, 69], [74, 68, 84], [105, 24, 102], [63, 92, 52], [87, 1], [88, 36, 1], [20, 84], [45, 102, 1], [36, 106, 44], [76, 44, 1], [27, 69, 44], [11, 44, 1], [90, 1], [68, 36], [24, 75, 88], [68, 2, 45], [52, 68], [27, 44, 74], [65, 73, 52], [24, 88, 105], [72, 92, 33], [53, 1], [92, 106], [62, 92], [11, 332], [27, 34, 74], [103, 44], [69, 107, 33], [75, 24, 69], [97, 102, 69], [14, 52, 68], [90, 105, 1], [45, 2], [107, 33], [107, 69, 76], [44, 103, 2], [90, 65], [74, 36], [65, 68, 52], [52, 20], [103, 106], [14, 1, 52], [44, 33, 2], [62, 69, 1], [65, 102, 68], [62, 73], [84, 45, 69], [92, 1

In [6]:
for combination in combinations:
    pairwise_synergies = fetch_synergies_from_file(combination)
    overall_response = analyze_combination_synergy(pairwise_synergies)
    overall_responses[tuple(combination)] = overall_response

save_overall_responses_to_csv(overall_responses, output_csv_path)

#Optionally, load and print the DataFrame for review
df = pd.read_csv(output_csv_path)
print(df.head())

  Drug Combination Overall Synergy Response
0           52, 45             Antagonistic
1           27, 69              Synergistic
2          76, 106             Antagonistic
3       68, 102, 2             Antagonistic
4          102, 45             Antagonistic
